In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline # <-- USAR Pipeline do sklearn AGORA
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight # <-- Importar para calcular pesos
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [17]:
# --- 1. Carregar e Preprocessar Dados (mantido como está) ---
df = pd.read_csv('/mnt/c/Users/AcerGamer/Downloads/Tuberculose/Projeto-Tuberculose/dados/arquivos_csv/dados_tuberculose_transformados.csv',low_memory=False, encoding='latin1')


In [18]:
# --- ETAPAS DE DEBUG PARA COLUNA 'FORMA' ---
print("--- Debug da Coluna 'FORMA' ---")
print(f"Valores únicos em 'FORMA' ANTES do mapeamento: {df['FORMA'].unique()}")
print(f"Contagem de nulos em 'FORMA' ANTES do mapeamento: {df['FORMA'].isnull().sum()}")


--- Debug da Coluna 'FORMA' ---
Valores únicos em 'FORMA' ANTES do mapeamento: [ 1.  2.  3. nan]
Contagem de nulos em 'FORMA' ANTES do mapeamento: 188


In [4]:
# Mapeamento do atributo 'FORMA' para os valores numéricos esperados
# Certifique-se de que este mapeamento está correto para seus dados
df['FORMA'] = df['FORMA'].map({
    'PULMONAR': 1.0,
    'EXTRAPULMONAR': 2.0,
    'MISTA': 3.0
})


In [5]:
print(f"Valores únicos em 'FORMA' APÓS o mapeamento: {df['FORMA'].unique()}")
print(f"Contagem de nulos em 'FORMA' APÓS o mapeamento: {df['FORMA'].isnull().sum()}")

Valores únicos em 'FORMA' APÓS o mapeamento: [nan]
Contagem de nulos em 'FORMA' APÓS o mapeamento: 502271


In [6]:
df.dropna(subset=['FORMA'], inplace=True)

In [7]:
print(f"Valores únicos em 'FORMA' APÓS dropna: {df['FORMA'].unique()}")
print(f"Contagem de nulos em 'FORMA' APÓS dropna: {df['FORMA'].isnull().sum()}")
print("--- Fim do Debug da Coluna 'FORMA' ---")

Valores únicos em 'FORMA' APÓS dropna: []
Contagem de nulos em 'FORMA' APÓS dropna: 0
--- Fim do Debug da Coluna 'FORMA' ---


In [8]:
# --- ADIÇÃO CRÍTICA: TRATAMENTO DE TIPOS E REMOÇÃO DE COLUNAS COM MUITOS NULOS ---

# 1. Lista de colunas com ALTÍSSIMA porcentagem de valores nulos (mais de 90-95%)
# Estas colunas raramente contribuem e podem ser removidas para simplificar
cols_to_drop_high_nulls = [
    'DT_TRANSRM', 'CS_FLXRET', 'FLXRECEBI', 'MIGRADO_W', 'ID_OCUPA_N',
    'TESTE_TUBE', 'BACILOS_E2', 'RIFAMPICIN', 'ISONIAZIDA', 'ETAMBUTOL',
    'ESTREPTOMI', 'PIRAZINAMI', 'ETIONAMIDA', 'OUTRAS', 'OUTRAS_DES',
    'TRAT_SUPER', 'DOENCA_TRA', 'DT_MUDANCA', 'SITUA_9_M', 'SITUA_12_M',
    'TRANSF', 'UF_TRANSF', 'MUN_TRANSF'
]
# Remover estas colunas do DataFrame
df = df.drop(columns=[col for col in cols_to_drop_high_nulls if col in df.columns])


In [9]:
# 2. Lista de colunas que são CATEGÓRICAS (incluindo as que são float mas representam códigos)
# Estas colunas serão explicitamente convertidas para string para o OneHotEncoder
categorical_cols_for_conversion = [
    'ID_AGRAVO', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N',
    'RAIOX_TORA', 'BACILOSC_E', 'CULTURA_ES', 'HIV', 'HISTOPATOL',
    'EXTRAPU1_N', 'EXTRAPU2_N', 'EXTRAPUL_O', 'AGRAVOUTRA', 'AGRAVOUTDE',
    'BACILOSC_O', 'CULTURA_OU', 'SG_UF_AT', 'BACILOSC_1', 'BACILOSC_2',
    'BACILOSC_3', 'BACILOSC_4', 'BACILOSC_5', 'BACILOSC_6',
    'TEST_SENSI', 'ANT_RETRO', 'BAC_APOS_6', 'BENEF_GOV',
    'TP_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA',
    'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS', 'TRATAMENTO', 'INSTITUCIO',
    'AGRAVAIDS', 'AGRAVALCOO', 'AGRAVDIABE', 'AGRAVDOENC', 'AGRAVDROGA', 'AGRAVTABAC',
    'TEST_MOLEC', 'NU_CONTATO', 'TRATSUP_AT', 'SITUA_ENCE', 'TPUNINOT',
    'POP_LIBER', 'POP_RUA', 'POP_SAUDE', 'POP_IMIG'
]

In [10]:
# Converter essas colunas para tipo string. np.nan se tornará 'nan' (string)
for col in categorical_cols_for_conversion:
    if col in df.columns: # Verifica se a coluna não foi removida no passo anterior
        df[col] = df[col].astype(str)

In [11]:
# Separar features (X) e target (y)
X = df.drop('FORMA', axis=1)
y = df['FORMA']



In [12]:
print(f"Valores únicos em 'FORMA' (target) antes do LabelEncoder: {y.unique()}")


Valores únicos em 'FORMA' (target) antes do LabelEncoder: []


In [13]:
# Codificar a variável target 'y' (seus rótulos numéricos para 0, 1, 2)
le = LabelEncoder()
y_encoded = le.fit_transform(y) # Agora y_encoded terá 0, 1, 2



In [14]:
# Exibir mapeamento do LabelEncoder
print("Mapeamento do LabelEncoder (ID Original -> Código Interno):")
for original_id, encoded_value in zip(le.classes_, le.transform(le.classes_)):
    print(f"ID Original {original_id} -> Código Interno {encoded_value}")



Mapeamento do LabelEncoder (ID Original -> Código Interno):


In [15]:
# Divisão em treino e teste (mantido como está)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

# Identificar colunas numéricas e categóricas (mantido como está)
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()



ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Pré-processamento (mantido como está)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])



In [ ]:
# --- 2. Calcular Pesos de Classe ---
# Contagem de classes em y_train (para referência e confirmação)
print("\nContagem de classes em y_train (códigos internos):")
print(pd.Series(y_train).value_counts())



In [ ]:
# Calcule os pesos das classes para lidar com o desbalanceamento
# 'balanced' ajusta os pesos inversamente proporcionais às frequências das classes
classes = np.unique(y_train) # 0, 1, 2
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)


In [ ]:
# Crie um dicionário de mapeamento código interno -> peso
class_weights = dict(zip(classes, class_weights_array))

print("\nPesos de classe calculados:")
for code, weight in class_weights.items():
    original_id = le.inverse_transform([code])[0]
    print(f"Classe Original {original_id} (Código Interno {code}): Peso = {weight:.2f}")



In [ ]:
# Crie um array de pesos para cada amostra no y_train
# Cada amostra receberá o peso correspondente à sua classe
sample_weights_train = np.array([class_weights[label] for label in y_train])


In [ ]:
# --- 3. Construção do Pipeline ---
#pipeline_final = Pipeline(steps=[ # <--- ATENÇÃO: Agora é sklearn.pipeline.Pipeline
#    ('preprocessor', preprocessor),
#    ('selector', SelectFromModel(XGBClassifier(n_estimators=50, random_state=42, use_label_encoder=False, eval_metric='mlogloss'))),
#    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss', tree_method='hist'))
#])
# --- 3. Construção do Pipeline ---
num_classes = len(np.unique(y_encoded)) # Obter o número de classes

pipeline_final = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('selector', SelectFromModel(XGBClassifier(
        n_estimators=50,
        random_state=42,
        use_label_encoder=False,
        eval_metric='mlogloss',
        objective='multi:softprob', # Definido explicitamente
        num_class=num_classes # Definido explicitamente
    ))),
    ('classifier', XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='mlogloss',
        tree_method='hist',
        objective='multi:softprob', # Definido explicitamente
        num_class=num_classes # Definido explicitamente
    ))
])

print("\nIniciando o treinamento do pipeline com XGBoost e pesos de classe...")
pipeline_final.fit(X_train, y_train, classifier__sample_weight=sample_weights_train)
print("Treinamento concluído com pesos de classe.")


In [ ]:
print("\nIniciando o treinamento do pipeline com XGBoost e pesos de classe...")
# Passar os pesos das amostras para o estimador final (classifier) no pipeline
pipeline_final.fit(X_train, y_train, classifier__sample_weight=sample_weights_train) # <--- AQUI A MUDANÇA CRÍTICA
print("Treinamento concluído com pesos de classe.")



In [ ]:
# --- 4. Avaliação do Modelo (mantido como está) ---
y_pred = pipeline_final.predict(X_test)

# Mapear as previsões e os valores reais de volta para os nomes descritivos
mapeamento_nomes_descritivos = {
    1.0: "Pulmonar",
    2.0: "Extrapulmonar",
    3.0: "Mista"
}

y_pred_original_ids = le.inverse_transform(y_pred)
y_test_original_ids = le.inverse_transform(y_test)

y_pred_nomes = [mapeamento_nomes_descritivos[idx] for idx in y_pred_original_ids]
y_test_nomes = [mapeamento_nomes_descritivos[idx] for idx in y_test_original_ids]

# Definir a ordem exata das classes para exibição (Pulmonar, Extrapulmonar, Mista)
class_display_order = ["Pulmonar", "Extrapulmonar", "Mista"] # Certifique-se de que esta é a ordem que você quer

In [ ]:
# --- 4. Avaliação do Modelo ---
y_pred = pipeline_final.predict(X_test)

mapeamento_nomes_descritivos = {
    1.0: "Pulmonar",
    2.0: "Extrapulmonar",
    3.0: "Mista"
}

y_pred_original_ids = le.inverse_transform(y_pred)
y_test_original_ids = le.inverse_transform(y_test)

y_pred_nomes = [mapeamento_nomes_descritivos[idx] for idx in y_pred_original_ids]
y_test_nomes = [mapeamento_nomes_descritivos[idx] for idx in y_test_original_ids]

class_display_order = ["Pulmonar", "Extrapulmonar", "Mista"]


In [ ]:
print("\nRelatório de Classificação com Nomes Descritivos (com Pesos de Classe):")
print(classification_report(y_test_nomes, y_pred_nomes, target_names=class_display_order))

cm_nomes = confusion_matrix(y_test_nomes, y_pred_nomes, labels=class_display_order)
print("\nMatriz de Confusão com Nomes Descritivos (com Pesos de Classe):")
print(cm_nomes)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_nomes, display_labels=class_display_order)
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão com Pesos de Classe')
plt.show()